# I first tried changing solver tolerance from 1f-4 to 5f-1 during testing

I kept everything else the same, and right after training is done I add the following code and run one extra round of testing. I only change up the solver tolerance here from the original 1f-4 used in training to 5f-1

    model.deq.args = (get_default_ssrootfind_solver(5f-1, 5f-1, LimitedMemoryBroydenSolver; device=gpu,
                           original_dims=(128, 1), batch_size=batch_size, maxiters=50),)
    test_time = time()
    test_loss, test_acc, test_nfe = loss_and_accuracy(model, testiter)
    test_time = time() - test_time
    println(test_time)
    println(test_loss)
    println(test_acc)
    push!(ret, test_acc)

In other words, the code above is added right before these two lines in the code

    close(lg_wandb)
    close(lg_term)

Running the process above for the vanilla and skip DEQs, I obtain the plot below. The two lines represent accuracy during validation, and I also plot the single point accuracy after changing the solver tolerance. The vanilla point is clearly very low, and the skip point is unseen because it is about the same accuracy as the training/validation.

![Imgur](https://i.imgur.com/52e9wM7.png)

It is worth clarifying that this plot is for applying mdeq to MNIST. Applying fully-connected to MNIST is not susceptible to the same change in solver tolerance. I think there are two interesting observations here:

* Changing the solver tolerance causes the model to breakdown in some mdeq models but not fully-connected models. In other words, only in certain mdeq models do we see different solvers finding different steady states
* Not all mdeq models break under solver change, though. skip mdeq somehow manages to resist the change in solver tolerance, whereas vanilla mdeq does not.


My hypothesis for why this happens:

* mdeq could be more sophisticated than fully-connected networks, in the sense that there are more steady states for the mdeq network. That is why the solver tolerance matters for the steady state.
* I think it's confusing why skip mdeq doesn't have this issue. I should also point out that it doesn't matter whether we add the regularization term, i.e. skip and skip_with_no_extra_params can both use cheaper solvers in test time even though it is trained with lower tolerance.